<a href="https://colab.research.google.com/github/cjk5642/Rainfall-Prediction-App/blob/main/src/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys, os
collin_src_path = "/content/drive/MyDrive/Classes/Spring 2022/DSCI 590: Time Series Analysis/Final Project/src"
sys.path.append(collin_src_path)

# imports for LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
try:
  from meteostat import Daily, Point
except ModuleNotFoundError:
  !pip install meteostat
  from meteostat import Daily, Point

from utils import WeatherData
wd = WeatherData()
torch.manual_seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

Mounted at /content/drive


'cpu'

In [3]:
temp = wd.ml_data.melt(id_vars = ["date", "tavg", "tmin", "tmax", "prcp", "snow", "wdir", "wspd", 'wpgt', 'pres', 'tsun'])
temp[temp.value != 0]

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,variable,value
68400,2018-01-01,0.7,-7.1,8.9,0.0,0.0,0.0,10.1,0.0,1025.9,0.0,ABQ,1
68401,2018-01-02,3.0,-4.3,12.8,0.0,0.0,0.0,7.6,0.0,1024.6,0.0,ABQ,1
68402,2018-01-03,2.8,-3.8,10.0,0.0,0.0,0.0,7.6,0.0,1027.5,0.0,ABQ,1
68403,2018-01-04,3.9,-0.5,11.1,0.0,0.0,0.0,4.0,0.0,1023.2,0.0,ABQ,1
68404,2018-01-05,3.3,-3.8,12.8,0.0,0.0,0.0,4.0,0.0,1025.5,0.0,ABQ,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7438875,2022-02-24,5.5,1.0,11.0,0.0,0.0,53.0,9.9,0.0,1022.6,0.0,VEF,1
7438876,2022-02-25,7.3,1.0,13.0,0.0,0.0,43.0,12.2,0.0,1023.6,0.0,VEF,1
7438877,2022-02-26,7.8,3.0,13.0,0.0,0.0,29.0,13.8,0.0,1029.3,0.0,VEF,1
7438878,2022-02-27,9.9,0.7,17.0,0.0,0.0,292.0,6.6,0.0,1028.2,0.0,VEF,1


# Sizes
## Input = 80
80

## Output
Want to produce 30 samples 

In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_size: int, hidden_size: int, num_layers: int = 2, dropout: float = 0):
    super(LSTM, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = 30
    self.num_layers = num_layers
    self.dropout = dropout

    self.lstm = nn.LSTM(input_size = self.input_size, 
                        hidden_size = self.hidden_size, 
                        num_layers = self.num_layers, 
                        dropout = self.dropout,
                        batch_first = True)
    self.out = nn.Linear(self.hidden_size, self.output_size, device=device)

  def forward(self, x):
    c0 = torch.randn(self.num_layers, x.size()[0], self.hidden_size, device = device)
    h0 = torch.randn(self.num_layers, x.size()[0], self.hidden_size, device = device)
    x, (self.hn, self.cn) = self.lstm(x, (h0, c0))
    x = F.relu(x)
    x = self.out(x)
    return x

In [ ]:
mat = torch.rand(1000, 10, 50)
lstm = LSTM(input_size = 50, hidden_size = 100, num_layers = 10)
x = lstm(mat)

In [ ]:
x.size()

torch.Size([1000, 10, 30])